# Strava sandbox

https://stackoverflow.com/questions/52880434/problem-with-access-token-in-strava-api-v3-get-all-athlete-activities

https://developers.strava.com/docs/authentication/

https://developers.strava.com/docs/getting-started/

In [26]:
import requests as r
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import time
import json

import os, sys

In [27]:
# globals for oauth pattern

email = 'max.schultz1@gmail.com'
password = 'Strava123'

client_id = '42763'
client_secret = '86c423e10bec9e9bdc577ea809d94b169e72d5e2'

redirect_uri = 'http://localhost/exchange_token'
response_type = 'code'
scope = 'activity:read_all'

In [28]:
auth_url = 'http://www.strava.com/oauth/authorize?client_id={client_id}&response_type={response_type}&redirect_uri={redirect_uri}&scope={scope}'.format(
    client_id = client_id,
    response_type = response_type,
    redirect_uri = redirect_uri,
    scope = scope
)
auth_url

'http://www.strava.com/oauth/authorize?client_id=42763&response_type=code&redirect_uri=http://localhost/exchange_token&scope=activity:read_all'

this will ask for login and auth, currently done in brower, move to api where possible

In [29]:
# attempt to automate in selenium

driver = webdriver.Chrome()
driver.get(auth_url)


time.sleep(1)
email_element = driver.find_element_by_name("email")
email_element.send_keys(email)

pw_element = driver.find_element_by_name("password")
pw_element.send_keys(password)

login_element = driver.find_element_by_id("login-button")
login_element.click()

time.sleep(1)
auth_element = driver.find_element_by_id("authorize")
auth_element.click()

resp_url = driver.current_url
driver.quit()

resp_url

'http://localhost/exchange_token?state=&code=0a48b4afeb8ad57beb2462c01988cadddb41a65f&scope=read,activity:read_all'

In [30]:
code = resp_url.split("code=")[1].split("&")[0]
code

'0a48b4afeb8ad57beb2462c01988cadddb41a65f'

In [31]:
# sample post
    
tok_url = 'https://www.strava.com/oauth/token?client_id={client_id}&client_secret={client_secret}&code={code}&grant_typeauthorization_code'.format(
    client_id = client_id,
    client_secret = client_secret,
    code = code
)
tok_url

'https://www.strava.com/oauth/token?client_id=42763&client_secret=86c423e10bec9e9bdc577ea809d94b169e72d5e2&code=0a48b4afeb8ad57beb2462c01988cadddb41a65f&grant_typeauthorization_code'

In [32]:
tok_resp = r.post(tok_url)
print(tok_resp.status_code)

tok_resp.json()

200


{'token_type': 'Bearer',
 'expires_at': 1594664135,
 'expires_in': 18323,
 'refresh_token': 'fa8a5826f5d2ae1dbb5459ebf9fbe34595b28d1b',
 'access_token': '7ace92314b000b46ec8fd04d3254117ad58e8800',
 'athlete': {'id': 46800838,
  'username': None,
  'resource_state': 2,
  'firstname': 'Max',
  'lastname': 'Schultz',
  'city': None,
  'state': None,
  'country': None,
  'sex': 'M',
  'premium': False,
  'summit': False,
  'created_at': '2019-09-24T08:30:50Z',
  'updated_at': '2020-07-13T12:15:17Z',
  'badge_type_id': 0,
  'profile_medium': 'avatar/athlete/medium.png',
  'profile': 'avatar/athlete/large.png',
  'friend': None,
  'follower': None}}

In [33]:
access_token = tok_resp.json()['access_token']
f_access_token = "access_token={access_token}".format(access_token = access_token)
f_access_token

'access_token=7ace92314b000b46ec8fd04d3254117ad58e8800'

In [34]:
%%time

activites_url = "https://www.strava.com/api/v3/activities"

df_activities = pd.DataFrame()

page = 1
while True:
    
    # get page of activities from Strava
    print(page)
    resp = r.get(activites_url + '?' + f_access_token + '&per_page=50' + '&page=' + str(page))
    _dict = resp.json()
    
    if len(_dict) == 0 :
        break
    
    else :
        df_activities = df_activities.append(pd.DataFrame(_dict)) 
        page += 1

print(df_activities.shape)
print(list(df_activities))
df_activities.head()

1
2
3
(76, 48)
['resource_state', 'athlete', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'workout_type', 'id', 'external_id', 'upload_id', 'start_date', 'start_date_local', 'timezone', 'utc_offset', 'start_latlng', 'end_latlng', 'location_city', 'location_state', 'location_country', 'start_latitude', 'start_longitude', 'achievement_count', 'kudos_count', 'comment_count', 'athlete_count', 'photo_count', 'map', 'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged', 'gear_id', 'from_accepted_tag', 'upload_id_str', 'average_speed', 'max_speed', 'has_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high', 'elev_low', 'pr_count', 'total_photo_count', 'has_kudoed']
Wall time: 4.48 s


,resource_state,athlete,name,distance,moving_time,elapsed_time,total_elevation_gain,type,workout_type,id,...,average_speed,max_speed,has_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,pr_count,total_photo_count,has_kudoed
0,2,"{'id': 46800838, 'resource_state': 1}",Afternoon Run,6705.7,2183,2903,71.6,Run,0,3739581614,...,3.072,6.4,False,False,False,95.8,77.9,1,0,False
1,2,"{'id': 46800838, 'resource_state': 1}",Low & Slow,7410.5,2478,3104,78.3,Run,0,3731401454,...,2.991,6.6,False,False,False,95.8,77.9,0,1,False
2,2,"{'id': 46800838, 'resource_state': 1}",Afternoon Run,5012.4,1529,1712,12.3,Run,0,3705748109,...,3.278,7.1,False,False,False,95.8,82.2,2,1,False
3,2,"{'id': 46800838, 'resource_state': 1}",W. Jack,7370.3,2386,3014,81.2,Run,0,3691497115,...,3.089,6.8,False,False,False,95.8,77.9,0,2,False
4,2,"{'id': 46800838, 'resource_state': 1}","Great 5km, atrocious 10km",10016.3,3327,4239,95.3,Run,0,3672419519,...,3.011,6.2,False,False,False,95.8,48.5,2,2,False


In [36]:
df_activities.to_csv("sandbox.csv")